In [1]:
%cd ..

/workspaces/Code


In [2]:
%matplotlib inline

from __future__ import annotations
import os
import numpy as np
import tensorflow as tf

from ModelBuilders import MobileNetBuilder, ModelOptions, BaseModelBuilder

from map_provider import ImageProvider, ImageProjection, MapProvider

from ExperimentRunners.RecallRunner import RecallRunnerOptions, RecalRunner, RecallSaverOptions, RecallSaver

from vector import Vector2D

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

def enable_gpu_memory_growth():
    """
    Enables memory growth mode for GPUs.
    """
    gpus = tf.config.experimental.list_physical_devices('GPU')
    assert len(gpus) > 0, "No GPUs detected!"
            
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

enable_gpu_memory_growth()

In [4]:
options = ModelOptions(
    builder_label="MobileNet",
    ingest_dense=64,
    output_dense=8,
    model_nn=35,
    trainable_from_index=35,
    epochs=1,
    batch_size=1)

model_builder = MobileNetBuilder(options)

In [5]:
def _experiment_folder(model_builder: BaseModelBuilder):
    options = model_builder.get_options()
    label = options.builder_label
    representation = options.representation()

    experiment_folder = f"Data/{label}/{representation}"
    os.makedirs(experiment_folder, exist_ok=True)

    return experiment_folder

def _load_recall_locations(drone_provider: MapProvider, location_count: int = 1500):
    save_folder = "Experiments/runtime_files/recall-locations"
    os.makedirs(save_folder, exist_ok=True)

    file = f"{save_folder}/size{location_count}-{drone_provider.projection}.npy"
    if os.path.isfile(file):
        return np.load(file)

    locations = drone_provider.generate_random_locations(location_count)
    np.save(file, locations)
    
    return locations

In [6]:
city_image = ImageProvider(image_path="City/NewCut/City_2017.jpg")
drone_image = ImageProvider(image_path="City/NewCut/City_2016.jpg")

projection = ImageProjection(
    position=Vector2D(8000, 8000),
    size=Vector2D(5000, 5000))

city_provider = MapProvider(
    image_provider=city_image,
    crop_size=672,
    projection=projection)

drone_provider = MapProvider(
    image_provider=drone_image,
    crop_size=672,
    projection=projection)

Loading image: City/NewCut/City_2017.jpg


In [ ]:
#21m 6 threaded

runner_options = RecallRunnerOptions(
    model_builder=model_builder,
    city_map=city_provider,
    drone_map=drone_provider,
    top=5,
    step_size=25,
    relevant_distance=50,
    drone_locations=_load_recall_locations(drone_provider, 1000))

runner = RecalRunner(runner_options)
stats = runner.run()

Processing recall locations: 100%|██████████| 1000/1000 [09:28<00:00,  1.76it/s]

In [ ]:
saver_options = RecallSaverOptions(
    city_map=city_provider,
    drone_map=drone_provider,
    save_samples=(4, 4))

saver = RecallSaver(_experiment_folder(model_builder), saver_options)
saver.save(stats)